<a href="https://colab.research.google.com/github/danielhcg/AntiSmishGPT/blob/main/llama_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install llamaapi

In [4]:
import json
import pandas as pd
from llamaapi import LlamaAPI

In [9]:
def evaluateMessage(sender, body, index, total):
    #initialize sdk
    llama = LlamaAPI('LLAMA-API-KEY')

    #to calculate progress of dataset processing
    progress=(index / total)*100
    print(f"Evaluating message {index}/{total} ({progress:.2f}%)")

    # Build the API request
    api_request_json = {
        "messages": [
            {"role": "user", "content": f"Is this message real or fake: '{body}' [Sender: {sender}]"}
        ],
        "functions": [
            {
                "name": "detect_fraudulent_sms",
                "description": "Analyze if an SMS message is legitimate or a scam",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "sender": {
                            "type": "string",
                            "description": "The sender of the SMS",
                        },
                        "body": {
                            "type": "string",
                            "description": "The full text of the SMS message",
                        }
                    },
                    "required": ["sender", "body"],
                }
            }
        ],
        "stream": False,
        "function_call": "detect_fraudulent_sms",
        "function_parameters": {
            "sender": sender,
            "body": body
        }
    }

    # Execute the Request
    response = llama.run(api_request_json)
    if response.status_code == 200:
        results = response.json()
        try:
            message_content = results['choices'][0]['message']['content']
            if message_content is not None:
                return "Real" if "real" in message_content.lower() else "Fake"
            else:
                return "Undetermined"  # Use Undetermined if the content is null or the function did not execute as expected
        except KeyError:
            return "Undetermined"  # KeyError handling
    else:
        return "Undetermined"  # Use Undetermined if the API call was not successful





In [10]:
def update_excel_with_results(file_path):
    # Load the Excel file
    df = pd.read_excel(file_path, header=None, names=['Sender', 'Body'])

    total = len(df)  # Total number of messages to process
    # Apply the evaluate_message function to each row with progress output
    results = [evaluateMessage(row['Sender'], row['Body'], i+1, total) for i, row in df.iterrows()]
    df['Result'] = results

    # Save the updated DataFrame back to the same Excel file
    df.to_excel(file_path, index=False, header=False)
    print("All messages have been evaluated and the results are saved back to the Excel file.")

In [11]:
# Example usage: specify the path to your Excel file that you want to update
update_excel_with_results("/content/testFile.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: '/content/testFile.xlsx'